In [1]:
from sentence_transformers import SentenceTransformer
import torch
from os import listdir
from os.path import isfile, join
import ast
import time
import numpy as np
from tqdm import tqdm

/mnt/nvme2/gnndataset/venv_v1/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BATCH = 1024
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
path = '../raw_dataset/nlp/'
files = [f for f in listdir(path) if (isfile(join(path, f)) and f.startswith('PaperAbstractsInvertedIndex.txt'))]

In [4]:
# Comment out to run for all the files
file = [files[0]]
file

['PaperAbstractsInvertedIndex.txt.2']

In [5]:
MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(MODEL).to(device)

In [6]:
def inv_idx_2_txt(data):
    abstract_inv = data["InvertedIndex"]
    total_len = int(data["IndexLength"])
    final_text = [""] * total_len
    for key, val in abstract_inv.items():
            for idx in val:
                final_text[idx] = key
    return " ".join(final_text)

In [ ]:
start = time.time()
paper_emb = {}
temp_paper_emb = []
paper_ids = []
counter = 0
index = 0
for filename in file: # Change to files if you follow previous comment 
    with open(join(path, filename)) as infile:
        for line in tqdm(infile):
            counter += 1
            cols = line.split("\t")
            data = ast.literal_eval(cols[1])
            paper_ids.append(int(cols[0]))
            temp_paper_emb.append([int(cols[0]), inv_idx_2_txt(data)])
            if counter % BATCH == 0:
                abstract = [data[1] for data in temp_paper_emb]
                emb = model.encode(abstract)
                for i, p in enumerate(paper_ids):
                    paper_emb[p] = emb[i]
                paper_ids = []
                temp_paper_emb = []
            if len(paper_emb) >= 10000000:
                index += 1
                #np.save("../IllinoisGraphDataset_v1/paper_384_emb_" + str(index) + ".npy", paper_emb)
                paper_emb = {}
time.time() - start

6397884it [2:39:26, 1141.21it/s]

In [ ]:
len(paper_emb)